In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
import transformers
from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from torch.utils.data import DataLoader

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,)
tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [5]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                            # attn_implementation="flash_attention_2",   #You can use flash attention on your local GPU with specific libraries
                                             )

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [11]:
import csv

def read_csv_to_pairs(file_path):
    pairs = []
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader)  # Skip the header row
        for row in csv_reader:
            if len(row) == 2:
                question, answer = row
                pairs.append((question, answer))
    return pairs

# Usage example
file_path = 'qa_1.txt'
qa_pairs = read_csv_to_pairs(file_path)
qa_pairs_snatch = read_csv_to_pairs('qa_2.txt')
qa_pairs_departed = read_csv_to_pairs('qa_3.txt')

In [13]:
qa_pairs.extend(qa_pairs_snatch)
qa_pairs.extend(qa_pairs_departed)
print(len(qa_pairs))
print(max(map(lambda x: len(x[1]), qa_pairs)))

270
210


In [14]:
def generate_prompt(user_query):
    sys_msg= "Respond to the question as Vinnie Jones."
    p =  "[INST]" + sys_msg +"\n"+ user_query['question'] + "[/INST]" +  user_query['answer']
    return p 

In [15]:
def generate_eval_prompt(p):
    sys_msg= "Respond to the question as Vinnie Jones."
    p =  "[INST]" + sys_msg +"\n"+ p + "[/INST]"
    return p 

In [16]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1


config = LoraConfig(r=LORA_R, lora_alpha=LORA_ALPHA, target_modules=[ "w1", "w2", "w3"], lora_dropout=LORA_DROPOUT, bias="none", task_type="CAUSAL_LM")
model = get_peft_model(model, config)



In [18]:
tokenize = lambda prompt: tokenizer(prompt, truncation=True, max_length=CUTOFF_LEN, padding="max_length")


In [19]:
def create_dataloader(qa_pairs, batch_size=8, shuffle=True):
    # Create a dataset from the qa_pairs
    dataset = Dataset.from_dict({'question': [pair[0] for pair in qa_pairs],
                                 'answer': [pair[1] for pair in qa_pairs]})

    # Create a dataset dictionary
    dataset_dict = DatasetDict({'train': dataset})

    # Create a dataloader from the dataset
    dataloader = DataLoader(dataset_dict['train'],
                            batch_size=batch_size,
                            shuffle=shuffle)
    return dataloader, dataset_dict


In [20]:
dataloader, ds = create_dataloader(qa_pairs, batch_size=4)

In [21]:
train_data = ds["train"]

In [23]:
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=["question" , "answer"])

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

In [24]:
train_data

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 270
})

In [25]:
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=6,
        learning_rate=1e-4,
        logging_steps=2,
        optim="adamw_torch",
        save_strategy="epoch",
        output_dir="mixtral-moe-lora-instruct-shapeskeare"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
2,5.937900
4,3.969200
6,3.280400
8,2.323000
10,1.953700
12,1.910100
14,1.785700
16,1.673600
18,1.730100
20,1.567300


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=402, training_loss=1.0348933196779508, metrics={'train_runtime': 6372.2319, 'train_samples_per_second': 0.254, 'train_steps_per_second': 0.063, 'total_flos': 1.1530663955634586e+17, 'train_loss': 1.0348933196779508, 'epoch': 5.96})

In [27]:
model.config.use_cache = True

In [44]:
eval_prompt = generate_eval_prompt("As i couldn't complete the code, Bes and Ardavan gave me the code to complete the project.")
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200, repetition_penalty=1.15)[0]))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s> [INST]Respond to the question as Vinnie Jones.
As i couldn't complete the code, Bes and Ardavan gave me the code to complete the project.[/INST]</s>I didn't ask for help from anyone else. I did it myself. You want something done right? Do it yourself! Stand on your own two feet. Show some guts![/INST]You think you can’t do it alone? Prove yourself wrong. Rise to the challenge. Be a bloody legend! No one owes you anything. Earn your place through hard work and determination. Now go out there and make it happen!</s>
